## JSON => DataFrame 저장하기

(1) 데이터 준비

In [3]:
import pandas as pd

In [5]:
file3 = '../DATA/nobel.json'

(2) JSON ==> DataFrame으로 지정

In [6]:
pd.read_json(file3)

,prizes
0,"{'year': '2019', 'category': 'chemistry', 'lau..."
1,"{'year': '2019', 'category': 'economics', 'lau..."
2,"{'year': '2019', 'category': 'literature', 'la..."
3,"{'year': '2019', 'category': 'peace', 'laureat..."
4,"{'year': '2019', 'category': 'physics', 'overa..."
...,...
641,"{'year': '1901', 'category': 'chemistry', 'lau..."
642,"{'year': '1901', 'category': 'literature', 'la..."
643,"{'year': '1901', 'category': 'peace', 'laureat..."
644,"{'year': '1901', 'category': 'physics', 'laure..."


In [7]:
nobelDF = pd.read_json(file3)
nobelDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prizes  646 non-null    object
dtypes: object(1)
memory usage: 5.2+ KB


(3) 데이터 확인

In [9]:
print(nobelDF.head(3), nobelDF.tail(3), sep = '\n\n')

                                              prizes
0  {'year': '2019', 'category': 'chemistry', 'lau...
1  {'year': '2019', 'category': 'economics', 'lau...
2  {'year': '2019', 'category': 'literature', 'la...

                                                prizes
643  {'year': '1901', 'category': 'peace', 'laureat...
644  {'year': '1901', 'category': 'physics', 'laure...
645  {'year': '1901', 'category': 'medicine', 'laur...


(4) 데이터 추출

In [12]:
# 행 추출 => iloc[n1, ...] / loc[]
type(nobelDF.iloc[0]), nobelDF.iloc[0]

(pandas.core.series.Series,
 prizes    {'year': '2019', 'category': 'chemistry', 'lau...
 Name: 0, dtype: object)

In [17]:
nobelDF.iloc[0]['prizes']

{'year': '2019',
 'category': 'chemistry',
 'laureates': [{'id': '976',
   'firstname': 'John',
   'surname': 'Goodenough',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '977',
   'firstname': 'M. Stanley',
   'surname': 'Whittingham',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '978',
   'firstname': 'Akira',
   'surname': 'Yoshino',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'}]}

In [15]:
oneData = nobelDF.iloc[0]['prizes']
type(oneData)

dict

In [16]:
oneData.keys(), oneData.items()

(dict_keys(['year', 'category', 'laureates']),
 dict_items([('year', '2019'), ('category', 'chemistry'), ('laureates', [{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}])]))

In [21]:
for k, v in oneData.items():
    print(f'[{k}]\n===>{v}')
    if k == 'laureates':
        for d in v:
            for k1, v1 in d.items():
                print('-', k1, v1)

[year]
===>2019
[category]
===>chemistry
[laureates]
===>[{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}]
- id 976
- firstname John
- surname Goodenough
- motivation "for the development of lithium-ion batteries"
- share 3
- id 977
- firstname M. Stanley
- surname Whittingham
- motivation "for the development of lithium-ion batteries"
- share 3
- id 978
- firstname Akira
- surname Yoshino
- motivation "for the development of lithium-ion batteries"
- share 3


- json 중첩 데이터의 다중 반복문 => 단일 리스트 형태 처리 : 평탄화 / 정규화

In [62]:
pd.json_normalize(oneData)

,year,category,laureates
0,2019,chemistry,"[{'id': '976', 'firstname': 'John', 'surname':..."


In [23]:
pd.json_normalize(oneData, record_path = 'laureates')

,id,firstname,surname,motivation,share
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3


In [24]:
pd.json_normalize(oneData, record_path = 'laureates', meta = ['year', 'category'])

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


- 모든 행에 대한 정규화 / 평탄화 진행

In [32]:
# 컬럼 읽기 => DF객체변수명['컬럼명'], DF객체변수명.컬럼명
# ===> 모든 행에 record_path에 해당하는 키(key)가 존재해야 한다.
# ===> 없는 행에 record_path에 해당하는 키 추가
for data in nobelDF.prizes:
    # print(type(data), data.keys(), end = '\n\n')
    # 해당 dict에 laureates 키가 없다면 추가 
    data.setdefault('laureates', [])

In [33]:
nobelDF2 = pd.json_normalize(nobelDF.prizes, record_path = 'laureates', meta = ['year', 'category'])

- (7) JSON의 다양한 방식의 DataFrame 저장하기 => orient 파라미터 

In [47]:
json_data = '''
[
    {"id" : "1", "name" : "sravan", "age" : 22},
    {"id" : "2", "name" : "harsha", "age" : 22},
    {"id" : "3", "name" : "deepika", "age" : 21},
    {"id" : "4", "name" : "jyothika", "age" : 23}

    ]
'''

In [48]:
type(json_data)

str

- orient = 'record' 설정 => JSON 파일데이터를 행 단위로 읽어서 DataFrame에 저장

In [49]:
df = pd.read_json(json_data, orient = 'records')
df

C:\Users\xnnly\AppData\Local\Temp\ipykernel_12124\764478428.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data, orient = 'records')


,id,name,age
0,1,sravan,22
1,2,harsha,22
2,3,deepika,21
3,4,jyothika,23


In [50]:
# 데이터 => 키 : {값}
json_data2 = '''
{
    "student-1" : {"id" : "1", "name" : "sravan", "age" : 22},
    "student-2" : {"id" : "2", "name" : "harsha", "age" : 22},
    "student-3" : {"id" : "3", "name" : "deepika", "age" : 22},
    "student-4" : {"id" : "4", "name" : "jyothika", "age" : 22}
}
'''

In [55]:
type(json_data2)

str

In [52]:
# 키 => 인덱스로 설정
df = pd.read_json(json_data2, orient = 'index')
df

C:\Users\xnnly\AppData\Local\Temp\ipykernel_12124\754430214.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data2, orient = 'index')


,id,name,age
student-1,1,sravan,22
student-2,2,harsha,22
student-3,3,deepika,22
student-4,4,jyothika,22


In [54]:
# 키 => 컬럼
df = pd.read_json(json_data2, orient = 'columns')  # orient 기본 인수는 'columns'
df

C:\Users\xnnly\AppData\Local\Temp\ipykernel_12124\658967763.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data2, orient = 'columns')


,student-1,student-2,student-3,student-4
id,1,2,3,4
name,sravan,harsha,deepika,jyothika
age,22,22,22,22


In [56]:
df.T

,id,name,age
student-1,1,sravan,22
student-2,2,harsha,22
student-3,3,deepika,22
student-4,4,jyothika,22


In [61]:
# JSON Format
json_data3 = '''
[
    [ "1", "sravan",22 ],
    [ "2", "harsha",22 ],
    [ "3", "deepika",21 ],
    [ "4", "jyothika",23 ]
]
'''

In [59]:
# json_data3에는 키는 없고 값만 있으므로 다음과 같이 출력된다.
df = pd.read_json(json_data3)
df

C:\Users\xnnly\AppData\Local\Temp\ipykernel_12124\2491556138.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data3)


,0,1,2
0,1,sravan,22
1,2,harsha,22
2,3,deepika,21
3,4,jyothika,23


In [60]:
# 위 결과와 동일하다.
df = pd.read_json(json_data3, orient = 'values')
df

C:\Users\xnnly\AppData\Local\Temp\ipykernel_12124\1162130222.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data3, orient = 'values')


,0,1,2
0,1,sravan,22
1,2,harsha,22
2,3,deepika,21
3,4,jyothika,23
